Thetf.data API of Tensorflow is a great way to build a pipeline for sending data to the GPU. Setting up data augmentation can be a bit tricky though. In this tutorial I will go through the steps of setting up a data augmentation pipeline. The table of contents for this post:

   - Rotation and flipping
   - Color augmentations
   - Zooming
   - All augmentations combined
   - Full code example

# Some sample data

To illustrate the different augmentation techniques we need some demo data. CIFAR10 is available through Tensorflow so that is an easy start. For illustration purposes I take the first 8 examples of CIFAR10 and build a dataset with this. In real world scenarios this would be replaced by your own data loading logic.

In [38]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()


In [39]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [40]:
dataset = tf.data.Dataset.from_tensor_slices((x_train[0:8] / 255).astype(np.float32))

In [41]:
print(x_train.shape,"-et-", y_train.shape)

(50000, 32, 32, 3) -et- (50000, 1)


With a simple function we can plot images from this dataset:

In [42]:
import numpy as np 
import matplotlib.pyplot as plt

In [43]:
def plot_images(datasets, n_images, samples_per_image):
    output = np.zeros((32*n_images, 32* samples_per_image,3))
    row = 0
    for image in datasets.repeat(samples_per_image).batch(n_images):
        output[:, row*32:(row+1)*32] = np.vstack(images.numpy())
        row += 1
    plt.figure()
    plt.imshow(output)
    plt.show()
    

# Implementing augmentations

To augment the dataset it can beneficial to make augmenter functions: a function that receives an image (a tf.Tensor) and returns a new augmented image. By defining functions for each augmentation operation we can easily attach them to datasets and control when they are evaluated.

With this basic recipe for an augmenter function we can implement the augmenters itself. Here I will show examples of:

   - Orientation (flipping and rotation)
   * Color augmentations (hue, saturation, brightness, contrast)
   * Zooming

## Rotation and flipping

One of the most simplest augmentations is rotating the image 90 degrees. For this we can use the rot90 function of Tensorflow. To get a new random rotation for each image we need to use a random function from Tensorflow itself. Random functions from Tensorflow are evaluated for every input, functions from numpy or basic python only once which would result in a static augmentation.

In [44]:
def rotate(x: tf.Tensor) -> tf.Tensor:
    """Rotation augmentation

    Args:
        x: Image

    Returns:
        Augmented image
    """

    # Rotate 0, 90, 180, 270 degrees
    return tf.image.rot90(x, tf.random_uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))


**Flipping** is another easy-to-implement augmentation. For these augmentations we do not have to use a random number generator as Tensorflow has a built-in function that does this for us: random_flip_left_right and random_flip_up_down.

In [45]:

def flip(x: tf.Tensor) -> tf.Tensor:
    """Flip augmentation

    Args:
        x: Image to flip

    Returns:
        Augmented image
    """
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)

    return x


In [46]:
# exemple of flipe : 
x = [[[1.0, 2.0, 3.0],
      [4.0, 5.0, 6.0]],
    [[7.0, 8.0, 9.0],
      [10.0, 11.0, 12.0]]]
tf.image.flip_left_right(x)

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[ 4.,  5.,  6.],
        [ 1.,  2.,  3.]],

       [[10., 11., 12.],
        [ 7.,  8.,  9.]]], dtype=float32)>

In [47]:
tf.image.random_flip_up_down(x)

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[ 7.,  8.,  9.],
        [10., 11., 12.]],

       [[ 1.,  2.,  3.],
        [ 4.,  5.,  6.]]], dtype=float32)>

# Color augmentations

**Color augmentations** are applicable to almost every image learning task. In Tensorflow there are four color augmentations readily available: **hue, saturation, brightness and contrast**. These functions only require a range and will result in an unique augmentation for each image.

In [48]:
def color(x: tf.Tensor) -> tf.Tensor:
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x


# Zooming

**Zooming** is a powerful augmentation that can make a network robust to (small) changes in object size. This augmentation is a bit harder to implement as there is no single function that performs this operation completely. The Tensorflow function crop_and_resize function comes close as it can crop an image and then resize it to an arbitrary size. The function requires a list of ‘crop boxes’ that contain normalized coordinates (between 0 and 1) for cropping.

In [49]:
""" def zoom(x: tf.Tensor) -> tf.Tensor:
   

    # Generate 20 crop settings, ranging from a 1% to 20% crop.
    scales = list(np.arange(0.8, 1.0, 0.01))
    boxes = np.zeros((len(scales), 4))

    for i, scale in enumerate(scales):
        x1 = y1 = 0.5 - (0.5 * scale)
        x2 = y2 = 0.5 + (0.5 * scale)
        boxes[i] = [x1, y1, x2, y2]

    def random_crop(img):
        # Create different crops for an image
        crops = tf.image.crop_and_resize([img], boxes=boxes, box_ind=np.zeros(len(scales)), crop_size=(32, 32))
        # Return a random crop
        return crops[tf.random_uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]


    choice = tf.random_uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)

    # Only apply cropping 50% of the time
    return tf.cond(choice < 0.5, lambda: x, lambda: random_crop(x))
"""

' def zoom(x: tf.Tensor) -> tf.Tensor:\n   \n\n    # Generate 20 crop settings, ranging from a 1% to 20% crop.\n    scales = list(np.arange(0.8, 1.0, 0.01))\n    boxes = np.zeros((len(scales), 4))\n\n    for i, scale in enumerate(scales):\n        x1 = y1 = 0.5 - (0.5 * scale)\n        x2 = y2 = 0.5 + (0.5 * scale)\n        boxes[i] = [x1, y1, x2, y2]\n\n    def random_crop(img):\n        # Create different crops for an image\n        crops = tf.image.crop_and_resize([img], boxes=boxes, box_ind=np.zeros(len(scales)), crop_size=(32, 32))\n        # Return a random crop\n        return crops[tf.random_uniform(shape=[], minval=0, maxval=len(scales), dtype=tf.int32)]\n\n\n    choice = tf.random_uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)\n\n    # Only apply cropping 50% of the time\n    return tf.cond(choice < 0.5, lambda: x, lambda: random_crop(x))\n'

# Augmenting the Dataset

With all functions defined we can combine them in to a single pipeline. Applying these functions to a Tensorflow Dataset is very easy using the map function. The map function takes a function and returns a new and augmented dataset. When this new dataset is evaluated, the data operations defined in the function will be applied to all elements in the set. Chaining map functions makes it very easy to iteratively add new data mapping operations, like augmentations.

In [ ]:
# Add augmentations
augmentations = [flip, color, rotate]

# Add the augmentations to the dataset
for f in augmentations:
    # Apply the augmentation, run 4 jobs in parallel.
    dataset = dataset.map(f, num_parallel_calls=3)

# Make sure that the values are still in [0, 1]
dataset = dataset.map(lambda x: tf.clip_by_value(x, 0, 1), num_parallel_calls=4)

plot_images(dataset, n_images=10, samples_per_image=15)


If applying all augmentations is a bit to much – which it is in the example above – it is also possible to only apply them to a certain percentage of the data. For this we can use the same approach as for the zooming augmentation: a combination of a tf.cond and tf.random_uniform call.